In [2]:
import numpy as np


In [3]:
with open('./np_output/movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle=True)
with open('./np_output/movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)
with open('./np_output/movie-xmasks.npy', 'rb') as f:
    Xmasks = np.load(f, allow_pickle=True)

In [4]:
labels.shape

(156060, 5)

In [5]:
Xmasks

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [6]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((Xids,labels,Xmasks))


2024-02-12 17:29:50.701206: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-12 17:29:50.746126: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 17:29:51.044710: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 17:29:51.044854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 17:29:51.104678: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [7]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.int64, name=None))>

In [8]:
def map_fun(input_ids, mask, labels):
    return {'input_ids':input_ids,
            'attention_mask':mask},labels

In [9]:
dataset = dataset.map(map_fun)
dataset

<_MapDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(5,), dtype=tf.float64, name=None)}, TensorSpec(shape=(512,), dtype=tf.int64, name=None))>

In [10]:
batch_size = 16

In [11]:
dataset = dataset.shuffle(10000).batch(batch_size,drop_remainder=True)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 5), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 512), dtype=tf.int64, name=None))>

In [12]:
split = 0.9

In [13]:
size = int((Xids.shape[0]/batch_size)*split)
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [14]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 5), dtype=tf.float64, name=None)},
 TensorSpec(shape=(16, 512), dtype=tf.int64, name=None))

In [15]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 5), dtype=tf.float64, name=None)},
 TensorSpec(shape=(16, 512), dtype=tf.int64, name=None))

In [16]:
ds = tf.data.experimental.load('train', element_spec=train_ds.element_spec)
ds.take(1)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 5), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 512), dtype=tf.int64, name=None))>